# Argparse

`argparse` is Python's built-in library that allows the creation of typical command-line interfaces for Python scripts. Findout more in the [specific section of the official documentation](https://docs.python.org/3/library/argparse.html).

In [2]:
import argparse

## Parse arguments

For experimentation purposes, it's easier to pass arguments to be parsed as a list - `ArgumentParser.parse_args` allows this option.

---

The following cell creates an `ArgumentParser`, adds an argument to it, and shows the result of parsing.

In [3]:
my_parser = argparse.ArgumentParser(description='Process some integers.')
my_parser.add_argument("--value")
ans = my_parser.parse_args(["--value", "10"])
ans

Namespace(value='10')

The `ArgumentParser.parse_args` returns a special object that stores all the values of the command-line arguments. The following cell shows its type:

In [6]:
type(ans)

argparse.Namespace

You can access the values for your arguments using the `.` syntax.

In [7]:
ans.value

'10'

## Print help

To print the `--help` message, use `ArgumentParser.print_help`.

---

The following cell shows an example of such usage.

In [4]:
my_parser = argparse.ArgumentParser(description='Process some integers.')
my_parser.add_argument("--value")
my_parser.print_help()

usage: ipykernel_launcher.py [-h] [--value VALUE]

Process some integers.

options:
  -h, --help     show this help message and exit
  --value VALUE


## Name with dash

When you have an argument with a multi-word name, it's typical to separate the words with a dash (e.g., `--my-argument`). However, in Python, variable and attribute names cannot contain dashes. Therefore, the `ArgumentParser` will convert the dashed name to a format that is valid in Python, typically by replacing dashes with underscores. For example, `--my-argument` would be accessible as `args.my_argument` in the Python code.

---

The following example adds an argument that has a dash in its name.

In [5]:
my_parser = argparse.ArgumentParser()
my_parser.add_argument("--value-dash")
ans = my_parser.parse_args(["--value-dash", "10"])
ans.value_dash

'10'

After parsing, in Python, you can access the obtained value using the `value_dash` name.

## `metavar`

<a href="https://docs.python.org/3/library/argparse.html#metavar">Official description</a>, to my mind, isn't really clear so here I present alternative description.

It's typical for CLI utilities to show in the help how to pass values to them. For example, `grep` has a `--label` option that takes a value, and it's help describes that you should use the syntax `--label=<passed value>` for it, which is just shown in the cell below. So here `LABEL` is a value that should be replaced with an argument during the call, and this type of constraint is called metavar.

In [6]:
%%bash
grep --help | grep label

      --label=LABEL         use LABEL as the standard input file name prefix


In argparse, every option that takes an argument has `metavar` as the upper case of `dest`, which is typical for cli units. But you can set your own using the `metavar` argument of the `add_argument` method. For arguments that take multiple values, you can set `metavar` as a tuple.

---

The following cell creates an `ArgumentParser` that uses metavars.

In [7]:
my_parser = argparse.ArgumentParser(description='Process some integers.')
my_parser.add_argument("--option")
my_parser.add_argument("--set_metavar", metavar="this_is_metavar")
my_parser.add_argument(
    "--double_metavar",
    nargs=2,
    metavar=('value1', 'value2')
)

_StoreAction(option_strings=['--double_metavar'], dest='double_metavar', nargs=2, const=None, default=None, type=None, choices=None, required=False, help=None, metavar=('value1', 'value2'))

This is what the `--help` of this programme looks like. Pay attention to the metavars of the options.

In [8]:
my_parser.print_help()

usage: ipykernel_launcher.py [-h] [--option OPTION]
                             [--set_metavar this_is_metavar]
                             [--double_metavar value1 value2]

Process some integers.

options:
  -h, --help            show this help message and exit
  --option OPTION
  --set_metavar this_is_metavar
  --double_metavar value1 value2


The following cell shows the results of the parsing.

In [9]:
my_parser.parse_args([
    "--option", "option_value", 
    "--set_metavar", "metavar_value", 
    "--double_metavar", "value1", "value2"
])

Namespace(option='option_value', set_metavar='metavar_value', double_metavar=['value1', 'value2'])

## Default values

There are two ways to specify a default value for an argument:

- Specify the `deafult` parameter for the `ArgumentParser.add_argument`.
- Use `ArgumentParser.set_deafults` method.

---

The following cell defines a parser that will be used as example.

In [53]:
my_parser = argparse.ArgumentParser()

The next code shows an argument that is added through the `add_argument` method with a specified `default`.

In [54]:
my_parser.add_argument("--arg", default='default')
my_parser.parse_args([])

Namespace(arg='default')

The empty `parse_args` call returned a namespace with `arg='default'`.

The following cell illustrates the approach of using `ArgumentParser.set_deafults`.

In [55]:
my_parser.set_defaults(arg="default from set_deafults")
my_parser.parse_args([])

Namespace(arg='default from set_deafults')

As specified in code default value of the `arg` argument is changed to `'default from set_defaults'`

### `set_deafults`

Take a closer look at the `set_defaults` approach, as it has an some important property: 

If you don't add an argument that is mentioned in the `set_defaults` method, it will be added automatically. However it won't be displayed in the `help` message.

---

The following cell creates a new parser and applies `set_defaults` to it.

In [56]:
my_parser = argparse.ArgumentParser()
my_parser.set_defaults(new_arg="new arg")

Any `parse_args` call results in a namespace containing `new_arg`:

In [58]:
my_parser.parse_args([])

Namespace(new_arg='new arg')

There are no any mentions of `new_arg` in the `print_help`:

In [59]:
my_parser.print_help()

usage: ipykernel_launcher.py [-h]

options:
  -h, --help  show this help message and exit


## Subparsers

Sometimes in your program, you have to define two different approaches to build process your input. For example, in machine learning, it's a common task to train different models with completely different sets of arguments in the same model building pipeline.

In that case, you must define a subparser that handles the parameters of the different models differently.

---

The following cell illustrates how to define a parser with subparsers.

In [60]:
my_parser = argparse.ArgumentParser()
subparsers = my_parser.add_subparsers(dest="submodule")

subparser1 = subparsers.add_parser(
    "subparser1", help="This is subparser1"
)
subparser1.add_argument("--arg_of_sp1")

subparser2 = subparsers.add_parser(
    "subparser2", help="This is subparser2"
)
subparser2.add_argument("--arg_of_sp2")
None

Help of the parser with subparsers is illustrated in the following cell.

In [61]:
my_parser.print_help()

usage: ipykernel_launcher.py [-h] {subparser1,subparser2} ...

positional arguments:
  {subparser1,subparser2}
    subparser1          This is subparser1
    subparser2          This is subparser2

options:
  -h, --help            show this help message and exit


The next code shows the kind of `arparse.Namespace` that you will get from the parser that defines supparser.

In [62]:
my_parser.parse_args(["subparser1", "--arg_of_sp1", "10"])

Namespace(submodule='subparser1', arg_of_sp1='10')